In [3]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [17]:
import json

# Open the 'Intent.json' file and load its contents as a JSON object
with open('Intent.json') as file:
    data = json.load(file)

# Initialize empty lists to store training sentences, labels, unique labels, and responses
training_sentences = []
training_labels = []
labels = []
responses = []

# Loop through each intent in the JSON data
for intent in data['intents']:
    # Loop through each pattern in the current intent's patterns
    for pattern in intent['patterns']:
        # Add the pattern to the list of training sentences
        training_sentences.append(pattern)
        # Add the intent's tag to the list of training labels
        training_labels.append(intent['tag'])

    # Add the list of responses for the current intent to the responses list
    responses.append(intent['responses'])

    # If the intent's tag is not already in the list of labels, add it
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

# Calculate the number of unique labels (classes)
num_classes = len(labels)


In [18]:

# Initialize the LabelEncoder
lbl_encoder = LabelEncoder()

# Fit the LabelEncoder with the training labels
lbl_encoder.fit(training_labels)

# Transform the training labels into encoded numeric values
training_labels = lbl_encoder.transform(training_labels)


In [19]:


# Define parameters for the tokenizer and padding
vocab_size = 1000         # Maximum number of words to keep in the vocabulary
embedding_dim = 16        # Dimension of the embedding vectors
max_len = 20              # Maximum length of the sequences
oov_token = "<OOV>"       # Token for out-of-vocabulary words

# Initialize the Tokenizer with the specified vocabulary size and OOV token
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)

# Fit the tokenizer on the training sentences
tokenizer.fit_on_texts(training_sentences)

# Retrieve the word index that maps words to their indices
word_index = tokenizer.word_index

# Convert the training sentences into sequences of integers
sequences = tokenizer.texts_to_sequences(training_sentences)

# Pad the sequences to ensure they all have the same length
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)


In [ ]:


# Define the model
model = Sequential()  #The Sequential model is a linear stack of layers.

# Add an Embedding layer
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))

# Add a GlobalAveragePooling1D layer
model.add(GlobalAveragePooling1D())

# Add the first Dense layer with ReLU activation
model.add(Dense(16, activation='relu'))

# Add the second Dense layer with ReLU activation
model.add(Dense(16, activation='relu'))

# Add the output Dense layer with softmax activation
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

# Set the number of epochs
epochs = 500

# Train the model
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)


In [21]:
import pickle
# Save the trained model
model.save("chat_model")  # Save the entire model to 'chat_model'

# Save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    # 'tokenizer' is saved to 'tokenizer.pickle'

# Save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)
    # 'lbl_encoder' is saved to 'label_encoder.pickle'


In [22]:
import colorama
colorama.init()
from colorama import Fore, Style
import random


# Load the intents data from a JSON file
with open("Intent.json") as file:
    data = json.load(file)

def chat():
    # Load the trained model from file
    model = keras.models.load_model('chat_model')

    # Load the tokenizer object from file
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # Load the label encoder object from file
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # Set the maximum length for input sequences
    max_len = 20

    while True:
        # Prompt user for input and display the prompt in light blue
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()

        # Exit the loop if the user types "quit"
        if inp.lower() == "quit":
            break

        # Convert user input to sequences and pad them to max_len
        result = model.predict(keras.preprocessing.sequence.pad_sequences(
            tokenizer.texts_to_sequences([inp]),
            truncating='post', maxlen=max_len))

        # Get the predicted tag from the result
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        # Find the appropriate response based on the predicted tag
        for i in data['intents']:
            if i['tag'] == tag:
                # Print the response in green color
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL, np.random.choice(i['responses']))

# Print the initial message in yellow to inform the user about starting the chat
print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)

# Call the chat function to start the interactive chat session
chat()


Start messaging with the bot (type quit to stop)!
User: hey there
1/1 [==============================] - 0s 81ms/step
ChatBot: Hello
User: who are you
1/1 [==============================] - 0s 38ms/step
ChatBot: I.m Joana, your bot assistant
User: i need help!
1/1 [==============================] - 0s 35ms/step
ChatBot: Yes Sure, How can I support you
User: i want to create account
1/1 [==============================] - 0s 40ms/step
ChatBot: Just go to our web site and follow the guidelines to create a new account
User: okay
1/1 [==============================] - 0s 36ms/step
ChatBot: Thanks
User: quit
